In [6]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="irony", lang="en")

loading configuration file https://huggingface.co/pysentimiento/bertweet-irony/resolve/main/config.json from cache at C:\Users\Jonas/.cache\huggingface\transformers\f8c3d0ff828a053eda9e09e31cffe4f34357b5f0db0a652e691a0cd8ddec65d7.cadb6579ef708993a8e8fe1203d7ec58dc0995c322918e64365f30f3f224a3c7
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/bertweet-irony",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "not ironic",
    "1": "ironic"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "ironic": 1,
    "not ironic": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id"

In [7]:
# taken from gpt.ipynb file
def calcFScore(truepos, falsepos, falseneg):
    FScoreResults = {}
    FScoreResults['precision'] = truepos/(truepos + falsepos)
    FScoreResults['recall'] = truepos/(truepos + falseneg)
    FScoreResults['F1'] = (2 * FScoreResults['precision'] * FScoreResults['recall'])/(FScoreResults['precision'] + FScoreResults['recall'])
    return FScoreResults

In [8]:
# Calculate the scores based on the answers from pysentimiento, this method is simply adapted from the other scores methods for GPT
def scoresSentimiento(dataset, resultColumn):
    truepos = 0
    falsepos = 0
    trueneg = 0
    falseneg = 0

    for index, row in dataset.iterrows():
        sentimientoEval = row[resultColumn]
        if (int(row[1]) == 1):
            if (int(sentimientoEval) == 1):
                truepos = truepos + 1
            else:
                falseneg = falseneg + 1
        elif (int(row[1]) == 0):
            if (int(sentimientoEval) == 1):
                falsepos = falsepos + 1
            else:
                trueneg = trueneg + 1

    numResults = calcFScore(truepos, falsepos, falseneg)
    numResults['tp'] = truepos
    numResults['fp'] = falsepos
    numResults['fn'] = falseneg
    numResults['tn'] = trueneg
    return numResults

In [11]:
import pandas as pd

datasetName = "fixedsetreadin"
datasetPath = "datasets\\" + datasetName + ".csv"
data = pd.read_csv(datasetPath)
data = data.head(100)

results = []

for index, row in data.iterrows():
    result = analyzer.predict(row[0])
    if(result.probas['ironic'] >= 0.5):
        results.append(1)
    else:
        results.append(0)

data['result'] = results
scores = scoresSentimiento(data, 2)
scores # F1 82/103

{'precision': 0.4864864864864865,
 'recall': 0.6206896551724138,
 'F1': 0.5454545454545455,
 'tp': 18,
 'fp': 19,
 'fn': 11,
 'tn': 52}

In [12]:
import pandas as pd

datasetName = "tweet_eval_irony_train"
datasetPath = "datasets\\" + datasetName + ".csv"
data = pd.read_csv(datasetPath)
data = data.head(100)

results = []

for index, row in data.iterrows():
    result = analyzer.predict(row[0])
    if(result.probas['ironic'] >= 0.5):
        results.append(1)
    else:
        results.append(0)

data['result'] = results
scores = scoresSentimiento(data, 2)
scores

{'precision': 0.7592592592592593,
 'recall': 0.8367346938775511,
 'F1': 0.796116504854369,
 'tp': 41,
 'fp': 13,
 'fn': 8,
 'tn': 38}

In [13]:
import pandas as pd

datasetName = "tweet_eval_irony_test"
datasetPath = "datasets\\" + datasetName + ".csv"
data = pd.read_csv(datasetPath)
data = data.head(100)

results = []

for index, row in data.iterrows():
    result = analyzer.predict(row[0])
    if(result.probas['ironic'] >= 0.5):
        results.append(1)
    else:
        results.append(0)

data['result'] = results
scores = scoresSentimiento(data, 2)
scores

{'precision': 0.6721311475409836,
 'recall': 0.9761904761904762,
 'F1': 0.7961165048543689,
 'tp': 41,
 'fp': 20,
 'fn': 1,
 'tn': 38}